<a href="https://colab.research.google.com/github/gift-framework/GIFT/blob/main/G2_ML/1_1c/K7_G2_TCS_v1_1c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/gift-framework/GIFT/blob/main/G2_ML/1.1/K7_G2_TCS_ExplicitMetric_v1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# K₇ G₂ TCS with Two-Speed LR + Complete GIFT 2.1 RG Flow - v1.1c

Complete TCS construction pipeline with:
- Full TCS geometry (M₁, Neck, M₂) with **extended neck** (σ_neck = 5.0)
- Neural φ-network with **torsion targeting** (not minimization)
- **Two-speed learning rate**: 1e-4 (phases 1-2), 5e-4 with warmup+cosine decay (phases 3-5)
- **Complete GIFT 2.1 RG flow** with all four components:
  - A·(∇·T): Torsion divergence (fixed calculation)
  - B·|T|²: Torsion norm
  - C·(∂ε g): Metric scale variation
  - D·fractality(T): Multi-scale structure
- Early RG calibration at epoch 2000 (after geometry stabilizes)
- Smart early stopping per phase
- Checkpoint system with automatic resumption
- Live Laplacian computation and harmonic extraction
- Full Yukawa tensor (21×21×77)

**v1.1c**: Fixes v1.1b training instability with progressive LR schedule while maintaining complete GIFT 2.1 formula.

**Goals**:
- Torsion: < 5% error (maintain v1.1 quality)
- Geometry: det(g) ≈ 2.0, positive definite
- RG Flow: Δα error < 20% (improve from v1.1b's 99%)
- Stable convergence: No oscillations, clean loss curves


## 1. Configuration and Imports


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from scipy.sparse import csr_matrix, lil_matrix, diags
from scipy.sparse.linalg import eigsh, spsolve
from scipy.spatial.transform import Rotation
import os
import json
import math
from pathlib import Path
from typing import Dict, Tuple, Optional, List

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

CONFIG = {
    'n_grid': 16,
    'n_grid_harmonics': 8,  # Reduced grid for harmonic extraction
    'n_fourier': 10,
    'hidden_dim': 256,
    'n_layers': 6,
    'batch_size': 1024,

    # TWO-SPEED LEARNING RATE (NEW v1.1c)
    'learning_rate_phase12': 1e-4,  # Phases 1-2: stabilize geometry
    'learning_rate_phase35': 5e-4,  # Phases 3-5: with warmup+decay
    'lr_warmup_epochs': 200,        # Warmup from 1e-4 to 5e-4
    'lr_min': 1e-5,                 # Minimum LR after decay

    'n_epochs_per_phase': 2000,
    'checkpoint_freq': 500,
    'checkpoint_dir': 'checkpoints_v1_1c',

    'tcs': {
        'r_neck_start': 0.35,
        'r_neck_end': 0.65,
        'r_acyl_cutoff': 10.0,
        'twist_angle': np.pi / 3,
        'neck_width': 5.0,  # NEW v1.1: Extended neck for RG flow accumulation
    },

    'target': {
        'det_g': 2.0,
        'torsion_norm': 0.0164,  # Target torsion magnitude from GIFT calibration
        'b2': 21,
        'b3': 77,
    },

    # NEW v1.1: Phase-specific torsion targets (gradual ramp-up)
    'torsion_targets': {
        1: 0.001,   # Phase 1: Establish geometry
        2: 0.005,   # Phase 2: ACyl matching
        3: 0.010,   # Phase 3: Cohomology refinement
        4: 0.015,   # Phase 4: Pre-calibration
        5: 0.0164   # Phase 5: Final GIFT target
    },

    'yukawa_samples': 200000,
    'torsion_threshold': 1e-6,
    'torsion_floor': 1e-9,

    'phases': {
        1: {
            'name': 'TCS_Neck',
            'weights': {
                'torsion': 1.0,      # NEW: replaces separate dphi/dpsi
                'det': 0.5,
                'positivity': 1.0,
                'neck_match': 2.0,
                'acyl': 0.0,
                'harmonicity': 0.0,
                'rg_flow': 0.0       # NEW v1.1
            },
            'early_stop': {         # NEW v1.1: Per-phase early stopping
                'patience': 200,
                'criteria': {
                    'neck_match': 1e-5,
                    'det': 1e-5,
                    'positivity': 1e-7
                }
            }
        },
        2: {
            'name': 'ACyl_Matching',
            'weights': {
                'torsion': 0.8,
                'det': 0.8,
                'positivity': 1.5,
                'neck_match': 0.5,
                'acyl': 0.5,
                'harmonicity': 0.0,
                'rg_flow': 0.0
            },
            'early_stop': {
                'patience': 200,
                'criteria': {
                    'acyl': 1e-5,
                    'det': 1e-5,
                    'torsion_target_reached': True  # Check if within 20% of target
                }
            }
        },
        3: {
            'name': 'Cohomology_Refinement',
            'weights': {
                'torsion': 0.6,
                'det': 0.5,
                'positivity': 1.0,
                'neck_match': 0.5,
                'acyl': 1.0,
                'harmonicity': 1.0,
                'rg_flow': 0.0
            },
            'early_stop': {
                'patience': 200,
                'criteria': {
                    'harmonicity': 1e-4,
                    'torsion_target_reached': True
                }
            }
        },
        4: {
            'name': 'Harmonic_Extraction',
            'weights': {
                'torsion': 0.5,
                'det': 1.0,
                'positivity': 1.0,
                'neck_match': 0.2,
                'acyl': 0.5,
                'harmonicity': 3.0,
                'rg_flow': 0.1       # Start introducing RG flow
            },
            'early_stop': {
                'patience': 200,
                'criteria': {
                    'harmonicity': 5e-5,
                    'torsion_target_reached': True
                }
            }
        },
        5: {
            'name': 'RG_Calibration',
            'weights': {
                'torsion': 0.3,      # Reduced from 3.0 to allow RG flow to dominate
                'det': 2.0,
                'positivity': 2.0,
                'neck_match': 0.1,
                'acyl': 0.2,
                'harmonicity': 2.0,
                'rg_flow': 0.05       # Full RG flow enforcement
            },
            'early_stop': {
                'patience': 500,
                'criteria': {
                    'torsion_target_reached': True,
                    'rg_flow_delta': 0.05,
                    'det': 1e-6,
                    'positivity': 1e-8,
                    'min_epochs': 1000
                }
            }
        },
    },

    # RG flow configuration (UPDATED v1.1c: early calibration)
    'rg_flow': {
        'lambda_max': 39.44,
        'target_delta_alpha': -0.9,
        'n_integration_steps': 100,
        'geodesic_batch_freq_base': 0.3,
        'calibration_epoch': 2000,  # UPDATED v1.1c: early calibration after geometry stable
        'adaptive_frequency': True,
        'monitor_components': True,
        'enable_divergence': True,
        'enable_epsilon_var': True,
        'enable_fractality': True,
    }
}

Path(CONFIG['checkpoint_dir']).mkdir(exist_ok=True)

print(f"Device: {device}")
print(f"Training grid: {CONFIG['n_grid']}^7")
print(f"Harmonics grid: {CONFIG['n_grid_harmonics']}^7")
print(f"Two-speed LR: phases 1-2 = {CONFIG['learning_rate_phase12']}, phases 3-5 = {CONFIG['learning_rate_phase35']} (with warmup+decay)")
print(f"RG calibration epoch: {CONFIG['rg_flow']['calibration_epoch']}")
print(f"Target: b₂={CONFIG['target']['b2']}, b₃={CONFIG['target']['b3']}")
print(f"Target torsion: ||T||={CONFIG['target']['torsion_norm']}")
print(f"Epochs per phase: {CONFIG['n_epochs_per_phase']}")
print(f"Phases: {len(CONFIG['phases'])}")
print(f"Extended neck width: σ_neck={CONFIG['tcs']['neck_width']}")
print(f"RG flow: λ_max={CONFIG['rg_flow']['lambda_max']}, Δα⁻¹={CONFIG['rg_flow']['target_delta_alpha']}")


Device: cuda
Training grid: 16^7
Harmonics grid: 8^7
Two-speed LR: phases 1-2 = 0.0001, phases 3-5 = 0.0005 (with warmup+decay)
RG calibration epoch: 2000
Target: b₂=21, b₃=77
Target torsion: ||T||=0.0164
Epochs per phase: 2000
Phases: 5
Extended neck width: σ_neck=5.0
RG flow: λ_max=39.44, Δα⁻¹=-0.9


## 2. Complete TCS Geometry with Extended Neck


In [ ]:
class ACylPotentials:
    def __init__(self, r_cutoff: float = 10.0):
        self.r_cutoff = r_cutoff

    def F(self, r: torch.Tensor) -> torch.Tensor:
        """ACyl potential F(r) → 1 as r → ∞."""
        return 1.0 - torch.exp(-r / self.r_cutoff)

    def H(self, r: torch.Tensor) -> torch.Tensor:
        """ACyl potential H(r) → 0 as r → ∞."""
        return torch.exp(-r / self.r_cutoff)

    def dF_dr(self, r: torch.Tensor) -> torch.Tensor:
        return torch.exp(-r / self.r_cutoff) / self.r_cutoff

    def dH_dr(self, r: torch.Tensor) -> torch.Tensor:
        return -torch.exp(-r / self.r_cutoff) / self.r_cutoff


class TCSGeometry:
    def __init__(self, config: Dict):
        self.n = config['n_grid']
        tcs_cfg = config['tcs']

        self.r_neck_start = tcs_cfg['r_neck_start']
        self.r_neck_end = tcs_cfg['r_neck_end']
        self.twist_angle = tcs_cfg['twist_angle']
        self.neck_width = tcs_cfg.get('neck_width', 5.0)  # NEW v1.1: Extended neck

        self.acyl = ACylPotentials(tcs_cfg['r_acyl_cutoff'])

        self.coords = np.linspace(0, 1, self.n, endpoint=False)

    def radial_coordinate(self, x: torch.Tensor) -> torch.Tensor:
        """Map T⁷ coordinate to radial parameter r ∈ [0,1]."""
        return x[:, 0]

    def region_classification(self, r: torch.Tensor) -> Dict[str, torch.Tensor]:
        """Classify points into M₁, Neck, M₂."""
        m1_mask = r < self.r_neck_start
        neck_mask = (r >= self.r_neck_start) & (r <= self.r_neck_end)
        m2_mask = r > self.r_neck_end

        return {'M1': m1_mask, 'Neck': neck_mask, 'M2': m2_mask}

    def neck_profile(self, r: torch.Tensor) -> torch.Tensor:
        """
        NEW v1.1: Extended Gaussian neck profile for RG flow accumulation.

        Args:
            r: Radial coordinate

        Returns:
            profile: Smooth interpolation across extended neck region
        """
        r_center = (self.r_neck_start + self.r_neck_end) / 2
        r_normalized = (r - r_center) / self.neck_width

        return torch.exp(-r_normalized**2 / 2)

    def neck_interpolation(self, r: torch.Tensor) -> torch.Tensor:
        """
        Smooth interpolation χ: 0 in M₁, 1 in M₂.
        NEW v1.1: Uses extended neck profile.
        """
        r_norm = (r - self.r_neck_start) / (self.r_neck_end - self.r_neck_start)
        r_norm = torch.clamp(r_norm, 0.0, 1.0)

        # Extended neck modulation
        profile = self.neck_profile(r)
        chi = 3 * r_norm**2 - 2 * r_norm**3

        # Smooth transition influenced by extended profile
        chi = chi * (1.0 + 0.5 * profile)

        return torch.clamp(chi, 0.0, 1.0)

    def twist_map(self, x: torch.Tensor) -> torch.Tensor:
        """Apply twist ψ on neck cross-section S¹×S¹."""
        r = self.radial_coordinate(x)
        chi = self.neck_interpolation(r)

        x_twisted = x.clone()

        theta1 = 2 * np.pi * x[:, 1]
        theta2 = 2 * np.pi * x[:, 2]

        theta1_new = theta1 + chi * self.twist_angle
        theta2_new = theta2 - chi * self.twist_angle

        x_twisted[:, 1] = (theta1_new / (2 * np.pi)) % 1.0
        x_twisted[:, 2] = (theta2_new / (2 * np.pi)) % 1.0

        return x_twisted

    def acyl_metric_correction(self, x: torch.Tensor, g_base: torch.Tensor) -> torch.Tensor:
        """Apply ACyl corrections to metric."""
        r = self.radial_coordinate(x)
        regions = self.region_classification(r)

        F = self.acyl.F(r).unsqueeze(-1).unsqueeze(-1)
        H = self.acyl.H(r).unsqueeze(-1).unsqueeze(-1)

        g_corrected = g_base.clone()

        g_corrected[regions['M1']] = g_base[regions['M1']] * (1.0 + 0.1 * H[regions['M1']])
        g_corrected[regions['M2']] = g_base[regions['M2']] * (1.0 + 0.1 * H[regions['M2']])

        return g_corrected

    def compute_normal_derivative_mismatch(self, phi_net: nn.Module, coords: torch.Tensor, extd) -> torch.Tensor:
        """Compute ACyl normal derivative matching condition."""
        r = self.radial_coordinate(coords)
        regions = self.region_classification(r)

        if not (regions['M1'].any() or regions['M2'].any()):
            return torch.tensor(0.0, device=coords.device)

        epsilon = 1e-4
        coords_plus = coords.clone()
        coords_plus[:, 0] += epsilon
        coords_plus[:, 0] = coords_plus[:, 0] % 1.0

        coords_minus = coords.clone()
        coords_minus[:, 0] -= epsilon
        coords_minus[:, 0] = coords_minus[:, 0] % 1.0

        phi = phi_net(coords)
        phi_plus = phi_net(coords_plus)
        phi_minus = phi_net(coords_minus)

        dphi_dr = (phi_plus - phi_minus) / (2 * epsilon)

        dF_dr = self.acyl.dF_dr(r).unsqueeze(-1).unsqueeze(-1).unsqueeze(-1)
        dH_dr = self.acyl.dH_dr(r).unsqueeze(-1).unsqueeze(-1).unsqueeze(-1)

        expected_derivative = phi * (0.1 * dH_dr)

        mismatch = torch.tensor(0.0, device=coords.device)
        if regions['M1'].any():
            mismatch += ((dphi_dr[regions['M1']] - expected_derivative[regions['M1']]) ** 2).mean()
        if regions['M2'].any():
            mismatch += ((dphi_dr[regions['M2']] - expected_derivative[regions['M2']]) ** 2).mean()

        return mismatch

geometry = TCSGeometry(CONFIG)
print(f"TCS regions: M₁ [0, {geometry.r_neck_start}], Neck [{geometry.r_neck_start}, {geometry.r_neck_end}], M₂ [{geometry.r_neck_end}, 1]")
print(f"Twist angle: {geometry.twist_angle:.4f} rad")
print(f"Extended neck width: σ_neck = {geometry.neck_width}")


TCS regions: M₁ [0, 0.35], Neck [0.35, 0.65], M₂ [0.65, 1]
Twist angle: 1.0472 rad
Extended neck width: σ_neck = 5.0


## 3. Enhanced φ-Network


In [ ]:
class FourierEncoding(nn.Module):
    def __init__(self, n_freqs: int):
        super().__init__()
        self.n_freqs = n_freqs
        self.register_buffer('freqs', 2 ** torch.arange(n_freqs, dtype=torch.float32))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x_proj = 2 * np.pi * x.unsqueeze(-1) * self.freqs
        return torch.cat([torch.sin(x_proj), torch.cos(x_proj)], dim=-1).flatten(-2)


class ResidualBlock(nn.Module):
    def __init__(self, dim: int):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, dim),
            nn.LayerNorm(dim),
            nn.GELU(),
            nn.Linear(dim, dim),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return x + self.net(x)


class PhiNetwork(nn.Module):
    def __init__(self, config: Dict):
        super().__init__()
        self.n_freqs = config['n_fourier']
        self.hidden = config['hidden_dim']
        self.n_layers = config['n_layers']

        self.encoding = FourierEncoding(self.n_freqs)
        input_dim = 7 * 2 * self.n_freqs

        self.input_proj = nn.Linear(input_dim, self.hidden)

        self.blocks = nn.ModuleList([
            ResidualBlock(self.hidden) for _ in range(self.n_layers)
        ])

        self.phi_head = nn.Sequential(
            nn.Linear(self.hidden, self.hidden // 2),
            nn.GELU(),
            nn.Linear(self.hidden // 2, 35)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        batch_size = x.shape[0]

        x_enc = self.encoding(x)
        h = self.input_proj(x_enc)

        for block in self.blocks:
            h = block(h)

        phi_flat = self.phi_head(h)

        phi = torch.zeros(batch_size, 7, 7, 7, device=x.device, dtype=x.dtype)

        idx = 0
        for i in range(7):
            for j in range(i+1, 7):
                for k in range(j+1, 7):
                    val = phi_flat[:, idx]

                    phi[:, i, j, k] = val
                    phi[:, i, k, j] = -val
                    phi[:, j, i, k] = -val
                    phi[:, j, k, i] = val
                    phi[:, k, i, j] = val
                    phi[:, k, j, i] = -val

                    idx += 1

        return phi

phi_net = PhiNetwork(CONFIG).to(device)
print(f"φ-network parameters: {sum(p.numel() for p in phi_net.parameters()):,}")


φ-network parameters: 866,083


## 4. G₂ Metric and Hodge Dual


In [ ]:
def compute_g2_metric(phi: torch.Tensor, epsilon: float = 1e-4, phase: int = 1) -> Tuple[torch.Tensor, torch.Tensor]:
    """Compute G₂ metric g_{ij} = (1/6) φ_{ipq} φ_j^{pq}."""
    batch_size = phi.shape[0]
    g = torch.zeros(batch_size, 7, 7, device=phi.device, dtype=phi.dtype)

    for i in range(7):
        for j in range(7):
            g[:, i, j] = (phi[:, i, :, :] * phi[:, j, :, :]).sum(dim=(-2, -1)) / 6.0

    # Force exact symmetrization
    g = (g + g.transpose(-2, -1)) / 2.0

    # Phase-adaptive regularization
    eps_adaptive = {1: 2e-3, 2: 1.5e-3, 3: 1e-3, 4: 5e-4, 5: 1e-4}
    eps = eps_adaptive.get(phase, epsilon)

    eye = torch.eye(7, device=phi.device, dtype=phi.dtype).unsqueeze(0)
    g = g + eps * eye

    g_inv = torch.linalg.inv(g)

    return g, g_inv


def normalize_metric(g: torch.Tensor, target_det: float = 2.0) -> torch.Tensor:
    """Normalize metric to have det(g) = target_det."""
    det = torch.linalg.det(g)
    scale = (target_det / det.abs().clamp(min=1e-8)) ** (1.0 / 7.0)
    return g * scale.unsqueeze(-1).unsqueeze(-1)


EPSILON_7D = None

def compute_hodge_dual(phi: torch.Tensor, g: torch.Tensor, g_inv: torch.Tensor) -> torch.Tensor:
    """Compute ψ = *φ."""
    global EPSILON_7D
    if EPSILON_7D is None:
        from itertools import permutations
        epsilon = torch.zeros(7, 7, 7, 7, 7, 7, 7)
        base = list(range(7))
        for perm in permutations(base):
            sign = 1
            temp = list(perm)
            for i in range(7):
                for j in range(i+1, 7):
                    if temp[i] > temp[j]:
                        sign *= -1
            epsilon[perm] = sign
        EPSILON_7D = epsilon.to(phi.device)

    batch_size = phi.shape[0]
    psi = torch.zeros(batch_size, 7, 7, 7, 7, device=phi.device, dtype=phi.dtype)

    det_g = torch.linalg.det(g)
    sqrt_det = torch.sqrt(det_g.abs().clamp(min=1e-8))

    phi_raised = torch.einsum('bijk,bil,bjm,bkn->blmn', phi, g_inv, g_inv, g_inv)

    for b in range(batch_size):
        psi[b] = torch.einsum('ijklmnp,mnp->ijkl', EPSILON_7D, phi_raised[b]) / (6.0 * sqrt_det[b])

    return psi

print("G₂ metric and Hodge dual ready")


G₂ metric and Hodge dual ready


## 5. Exterior Derivatives


In [ ]:
class ExteriorDerivative:
    def __init__(self, n_grid: int, epsilon: float = 1e-4):
        self.n = n_grid
        self.dx = 1.0 / n_grid
        self.epsilon = epsilon

    def compute_jacobian(self, phi_net: nn.Module, coords: torch.Tensor) -> torch.Tensor:
        """Compute Jacobian ∂φ_{ijk}/∂x^l."""
        batch_size = coords.shape[0]
        jacobian = torch.zeros(batch_size, 7, 7, 7, 7, device=coords.device, dtype=coords.dtype)

        for l in range(7):
            coords_plus = coords.clone()
            coords_plus[:, l] += self.epsilon
            coords_plus[:, l] = coords_plus[:, l] % 1.0

            coords_minus = coords.clone()
            coords_minus[:, l] -= self.epsilon
            coords_minus[:, l] = coords_minus[:, l] % 1.0

            phi_plus = phi_net(coords_plus)
            phi_minus = phi_net(coords_minus)

            jacobian[:, :, :, :, l] = (phi_plus - phi_minus) / (2 * self.epsilon)

        return jacobian

    def d_phi(self, jacobian: torch.Tensor) -> torch.Tensor:
        """Compute dφ from Jacobian."""
        batch_size = jacobian.shape[0]
        dphi = torch.zeros(batch_size, 7, 7, 7, 7, device=jacobian.device, dtype=jacobian.dtype)

        for i in range(7):
            for j in range(i+1, 7):
                for k in range(j+1, 7):
                    for l in range(7):
                        if l in {i, j, k}:
                            continue

                        val = (jacobian[:, j, k, l, i] -
                               jacobian[:, i, k, l, j] +
                               jacobian[:, i, j, l, k] -
                               jacobian[:, i, j, k, l])

                        indices = [i, j, k, l]
                        for perm_idx, perm in enumerate([(0,1,2,3), (0,1,3,2), (0,2,1,3), (0,2,3,1)]):
                            p = [indices[perm[m]] for m in range(4)]
                            sign = 1
                            for a in range(4):
                                for b in range(a+1, 4):
                                    if p[a] > p[b]:
                                        sign *= -1
                            dphi[:, p[0], p[1], p[2], p[3]] = sign * val

        return dphi

    def d_psi_norm(self, psi: torch.Tensor, phi_net: nn.Module, coords: torch.Tensor) -> torch.Tensor:
        """Compute ||dψ||."""
        jacobian = self.compute_jacobian(phi_net, coords)
        dphi_norm = (jacobian ** 2).sum(dim=(-4, -3, -2, -1))
        return torch.sqrt(dphi_norm.mean())

extd = ExteriorDerivative(CONFIG['n_grid'])
print("Exterior derivative operators ready")


Exterior derivative operators ready


## 6. NEW v1.1: Torsion Targeting Loss (CRITICAL FIX)


## 9. GIFT 2.1 RG Flow Components (NEW v1.1c)

Complete implementation of all four GIFT 2.1 terms with fixed divergence calculation.


In [ ]:
def compute_torsion_divergence(torsion: torch.Tensor, coords: torch.Tensor,
                                dx: float = 1.0/16) -> torch.Tensor:
    """
    FIXED v1.1c: Compute ∇·T using spatial variation as proxy for divergence.

    Simplified divergence: measures how torsion components vary across the batch.

    Args:
        torsion: (batch, 7, 7, 7, 7) - T^ijkl 4-form components (dphi)
        coords: (batch, 7) - coordinates
        dx: grid spacing

    Returns:
        div_T: (batch,) - scalar divergence measure
    """
    batch_size = torsion.shape[0]
    div_T = torch.zeros(batch_size, device=torsion.device)

    # Use component variation as proxy for divergence
    # Sum over all 4 indices properly
    for i in range(7):
        for j in range(7):
            for k in range(7):
                for l in range(7):
                    if batch_size > 1:
                        # Measure how much this component varies across batch
                        component = torsion[:, i, j, k, l]  # shape: (batch,)
                        component_var = torch.abs(component - component.mean())
                        div_T += component_var / dx

    return div_T / (7**4)  # Normalize by 7^4 total components

print("Torsion divergence function ready (FIXED v1.1c: handles 4-form correctly)")

Torsion divergence function ready (FIXED v1.1c: handles 4-form correctly)


In [ ]:
def compute_epsilon_derivative(phi_net: nn.Module, coords: torch.Tensor,
                               geometry, epsilon_0: float = 0.125) -> torch.Tensor:
    """
    NEW v1.1c: Compute ∂ε g = metric derivative w.r.t. scale ε.

    Measures how metric changes with RG scale (ε₀ = 1/8 from GIFT symmetry breaking).

    Args:
        phi_net: Neural network
        coords: (batch, 7)
        geometry: TCSGeometry object
        epsilon_0: GIFT scale (1/8)

    Returns:
        deps_g: (batch, 3) - [trace variation, det variation, norm variation]
    """
    delta_eps = 1e-4

    with torch.no_grad():
        # Baseline metric at ε₀
        phi_base = phi_net(coords)
        g_base, _ = compute_g2_metric(phi_base, phase=5)
        g_base = geometry.acyl_metric_correction(coords, g_base)

        # Perturbed metric (simulate scale change via coordinate rescaling)
        coords_scaled = coords * (1 + delta_eps / epsilon_0)
        phi_scaled = phi_net(coords_scaled % 1.0)
        g_scaled, _ = compute_g2_metric(phi_scaled, phase=5)
        g_scaled = geometry.acyl_metric_correction(coords_scaled % 1.0, g_scaled)

        # Compute variations
        trace_var = (torch.diagonal(g_scaled, dim1=-2, dim2=-1).sum(-1) -
                    torch.diagonal(g_base, dim1=-2, dim2=-1).sum(-1)) / delta_eps
        det_var = (torch.linalg.det(g_scaled) - torch.linalg.det(g_base)) / delta_eps
        norm_var = ((g_scaled**2).sum((-2,-1)) - (g_base**2).sum((-2,-1))) / delta_eps

    return torch.stack([trace_var, det_var, norm_var], dim=-1)

print("Epsilon derivative function ready (∂ε g term)")


Epsilon derivative function ready (∂ε g term)


In [ ]:
def compute_fractality_fourier(torsion: torch.Tensor) -> torch.Tensor:
    """
    NEW v1.1c: Fourier power spectrum slope as fractality measure.

    Fractal structures have power law: P(k) ~ k^(-α)
    Returns normalized α ∈ [0, 1] indicating multi-scale structure.

    Args:
        torsion: (batch, 7, 7, 7) - torsion components

    Returns:
        frac_idx: (batch,) - fractality index [0,1]
    """
    batch_size = torsion.shape[0]
    frac_idx = torch.zeros(batch_size, device=torsion.device)

    for b in range(batch_size):
        # Flatten torsion to 1D signal
        T_flat = torsion[b].flatten()

        # Skip if too small
        if len(T_flat) < 10:
            continue

        # FFT power spectrum
        fft = torch.fft.rfft(T_flat)
        power = torch.abs(fft)**2

        if len(power) < 3:
            continue

        # Log-log fit: log(P) = -α·log(k) + const
        k = torch.arange(1, len(power), device=torsion.device, dtype=torch.float32)
        log_k = torch.log(k + 1e-10)
        log_P = torch.log(power[1:] + 1e-10)

        # Linear regression for slope
        k_mean = log_k.mean()
        P_mean = log_P.mean()
        numerator = ((log_k - k_mean) * (log_P - P_mean)).sum()
        denominator = ((log_k - k_mean)**2).sum()

        if denominator > 1e-10:
            slope = numerator / denominator
            # Normalize: typical fractals have α ∈ [1, 3], map to [0, 1]
            frac_idx[b] = torch.clamp(-slope / 3.0, 0.0, 1.0)

    return frac_idx

print("Fractality function ready (Fourier spectrum method)")


Fractality function ready (Fourier spectrum method)


In [ ]:
class RGFlowGIFT:
    """
    NEW v1.1c: Complete GIFT 2.1 RG flow calculator.

    Implements: ℱ_RG = A·(∇·T) + B·|T|² + C·(∂ε g) + D·fractality(T)
                Δα = ∫ ℱ_RG dx
    """

    def __init__(self, config: Dict):
        self.config = config['rg_flow']
        self.epsilon_0 = 0.125  # 1/8 GIFT symmetry breaking scale

        # Learnable coefficients (will be calibrated at epoch 2000)
        self.A = -4.68   # divergence
        self.B = 15.17   # norm
        self.C = torch.tensor([10.0, 5.0, 1.0])  # epsilon derivatives
        self.D = 2.5     # fractality

        self.history = []  # for monitoring

    def compute_delta_alpha(self, phi_net, geometry, coords, torsion, epoch):
        """
        NEW v1.1c: Full GIFT 2.1 calculation with all four components.

        Args:
            phi_net: Neural network
            geometry: TCSGeometry
            coords: Sample coordinates
            torsion: Torsion tensor (dphi)
            epoch: Current epoch (for logging)

        Returns:
            delta_alpha: RG running value
            components: Dict with breakdown of each term
        """
        # Component A: divergence (if enabled)
        A_term = torch.tensor(0.0, device=coords.device)
        div_T_mean = 0.0
        if self.config.get('enable_divergence', True):
            div_T = compute_torsion_divergence(torsion, coords)
            div_T_mean = div_T.mean().item()
            A_term = self.A * div_T.mean()

        # Component B: norm (existing from v1.1)
        B_term = self.B * torch.norm(torsion)**2

        # Component C: epsilon variation (if enabled)
        C_term = torch.tensor(0.0, device=coords.device)
        if self.config.get('enable_epsilon_var', True):
            deps_g = compute_epsilon_derivative(phi_net, coords, geometry, self.epsilon_0)
            C_term = torch.dot(self.C.to(coords.device), deps_g.mean(0))

        # Component D: fractality (if enabled)
        D_term = torch.tensor(0.0, device=coords.device)
        frac_idx_mean = 0.0
        if self.config.get('enable_fractality', True):
            frac_idx = compute_fractality_fourier(torsion)
            frac_idx_mean = frac_idx.mean().item()
            D_term = self.D * frac_idx.mean()

        # Total integrand
        integrand = A_term + B_term + C_term + D_term

        # Geodesic integration over λ ∈ [0, lambda_max]
        lambdas = torch.linspace(0, self.config['lambda_max'],
                                self.config['n_integration_steps'], device=coords.device)
        delta_alpha = torch.trapz(integrand * torch.ones_like(lambdas), lambdas)

        # Log components for monitoring
        components = {
            'A_divergence': A_term.item() if torch.is_tensor(A_term) else A_term,
            'B_norm': B_term.item(),
            'C_epsilon': C_term.item() if torch.is_tensor(C_term) else C_term,
            'D_fractality': D_term.item() if torch.is_tensor(D_term) else D_term,
            'total': delta_alpha.item(),
            'div_T_mean': div_T_mean,
            'frac_idx_mean': frac_idx_mean,
        }

        if self.config.get('monitor_components', False):
            self.history.append({'epoch': epoch, **components})

        return delta_alpha, components

rg_flow_gift = RGFlowGIFT(CONFIG)
print("RGFlowGIFT class ready (complete GIFT 2.1 formula)")
print(f"  Initial coefficients: A={rg_flow_gift.A:.2f}, B={rg_flow_gift.B:.2f}, D={rg_flow_gift.D:.2f}")
print(f"  Component C weights: {rg_flow_gift.C.numpy()}")


RGFlowGIFT class ready (complete GIFT 2.1 formula)
  Initial coefficients: A=-4.68, B=15.17, D=2.50
  Component C weights: [10.  5.  1.]


In [ ]:
class SmartEarlyStopping:
    """
    NEW v1.1c: Smart early stopping with NaN detection and RG flow awareness.

    Prevents premature stopping while allowing phases to converge naturally.
    """

    def __init__(self, phase_config, phase_num):
        self.patience = phase_config.get('early_stop', {}).get('patience', 200)
        self.criteria = phase_config.get('early_stop', {}).get('criteria', {})
        self.min_epochs = self.criteria.get('min_epochs', 0)
        self.phase_num = phase_num

        self.counter = 0
        self.best_loss = float('inf')

    def check(self, epoch, losses, metrics, config):
        """
        Check if should stop training.

        Returns True if should stop, False otherwise.
        """
        # NaN check
        for key, val in losses.items():
            if isinstance(val, torch.Tensor):
                if torch.isnan(val) or torch.isinf(val):
                    print(f"⚠️  NaN/Inf detected in {key} at epoch {epoch}, stopping phase {self.phase_num}")
                    return True

        # Minimum epochs
        if epoch < self.min_epochs:
            return False

        # Check criteria satisfaction
        all_met = True
        for criterion, threshold in self.criteria.items():
            if criterion == 'min_epochs':
                continue
            elif criterion == 'torsion_target_reached':
                torsion_error = abs(metrics.get('actual_torsion', 0) - config['torsion_targets'][self.phase_num])
                torsion_error_rel = torsion_error / config['torsion_targets'][self.phase_num]
                if torsion_error_rel > 0.2:  # 20% tolerance
                    all_met = False
            elif criterion == 'rg_flow_delta' and self.phase_num >= 5:
                delta_alpha = metrics.get('delta_alpha', 0)
                target = config['rg_flow']['target_delta_alpha']
                rg_error_rel = abs(delta_alpha - target) / abs(target)
                if rg_error_rel > threshold:
                    all_met = False
            elif criterion in losses:
                loss_val = losses[criterion]
                if isinstance(loss_val, torch.Tensor):
                    loss_val = loss_val.item()
                if loss_val > threshold:
                    all_met = False

        if not all_met:
            self.counter = 0
            return False

        # Patience mechanism
        current_loss = losses.get('total', float('inf'))
        if isinstance(current_loss, torch.Tensor):
            current_loss = current_loss.item()

        if current_loss < self.best_loss * 0.999:  # 0.1% improvement
            self.best_loss = current_loss
            self.counter = 0
        else:
            self.counter += 1

        if self.counter >= self.patience:
            print(f"✓ Early stop phase {self.phase_num} at epoch {epoch}: criteria met for {self.patience} epochs")
            return True

        return False

print("SmartEarlyStopping class ready")


SmartEarlyStopping class ready


In [ ]:
class RGFlowMonitor:
    """
    NEW v1.1c: Detailed RG flow component tracking.

    Logs all GIFT 2.1 components (A, B, C, D) separately for analysis.
    """

    def __init__(self):
        self.history = []

    def log(self, epoch, rg_components, metrics):
        """Log RG flow components for this epoch."""
        entry = {
            'epoch': epoch,
            'delta_alpha': rg_components.get('total', 0),
            'A_div': rg_components.get('A_divergence', 0),
            'B_norm': rg_components.get('B_norm', 0),
            'C_eps': rg_components.get('C_epsilon', 0),
            'D_frac': rg_components.get('D_fractality', 0),
            'div_T': rg_components.get('div_T_mean', 0),
            'frac_idx': rg_components.get('frac_idx_mean', 0),
            'torsion_norm': metrics.get('actual_torsion', 0),
            'det_g': metrics.get('det_g', 0),
        }
        self.history.append(entry)

    def save(self, path='rg_flow_log.csv'):
        """Save component history to CSV."""
        import pandas as pd
        if len(self.history) > 0:
            df = pd.DataFrame(self.history)
            df.to_csv(path, index=False)
            print(f"RG flow component log saved to {path}")

rg_monitor = RGFlowMonitor()
print("RGFlowMonitor class ready")


RGFlowMonitor class ready


## 10. Two-Speed Learning Rate Scheduler (NEW v1.1c)

Progressive LR schedule to stabilize geometry before activating RG flow.


In [ ]:
class TwoSpeedLRScheduler:
    """
    NEW v1.1c: Two-speed learning rate with warmup and cosine decay.

    Schedule:
    - Phases 1-2: Constant 1e-4 (stabilize geometry)
    - Phase 3 start: Warmup from 1e-4 to 5e-4 over 200 epochs
    - Phases 3-5: Cosine decay from 5e-4 to 1e-5
    """

    def __init__(self, optimizer, config):
        self.optimizer = optimizer
        self.lr_phase12 = config['learning_rate_phase12']
        self.lr_phase35 = config['learning_rate_phase35']
        self.lr_min = config['lr_min']
        self.warmup_epochs = config['lr_warmup_epochs']

        # Track phase 3 start for warmup
        self.phase3_start_epoch = None
        self.current_phase = 1

    def get_lr(self, epoch, phase):
        """Compute LR for current epoch and phase."""
        if phase <= 2:
            # Phases 1-2: constant low LR
            return self.lr_phase12
        else:
            # Phases 3-5: warmup + cosine decay
            if self.phase3_start_epoch is None:
                # Mark start of phase 3
                self.phase3_start_epoch = epoch

            epochs_since_phase3 = epoch - self.phase3_start_epoch

            if epochs_since_phase3 < self.warmup_epochs:
                # Warmup: linear from lr_phase12 to lr_phase35
                alpha = epochs_since_phase3 / self.warmup_epochs
                lr = self.lr_phase12 + alpha * (self.lr_phase35 - self.lr_phase12)
            else:
                # Cosine decay from lr_phase35 to lr_min
                # Assuming total ~6000 epochs for phases 3-5
                decay_epochs = epochs_since_phase3 - self.warmup_epochs
                max_decay_epochs = 6000  # Approximate total for phases 3-5
                progress = min(decay_epochs / max_decay_epochs, 1.0)
                lr = self.lr_min + 0.5 * (self.lr_phase35 - self.lr_min) * (1 + math.cos(math.pi * progress))

            return lr

    def step(self, epoch, phase):
        """Update optimizer LR."""
        if phase != self.current_phase:
            self.current_phase = phase
            if phase == 3 and self.phase3_start_epoch is None:
                self.phase3_start_epoch = epoch

        lr = self.get_lr(epoch, phase)
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = lr

        return lr

print("TwoSpeedLRScheduler class ready")
print(f"  Phase 1-2: LR = {CONFIG['learning_rate_phase12']}")
print(f"  Phase 3-5: LR = {CONFIG['learning_rate_phase35']} with warmup over {CONFIG['lr_warmup_epochs']} epochs")
print(f"  Cosine decay to: LR_min = {CONFIG['lr_min']}")


TwoSpeedLRScheduler class ready
  Phase 1-2: LR = 0.0001
  Phase 3-5: LR = 0.0005 with warmup over 200 epochs
  Cosine decay to: LR_min = 1e-05


In [ ]:
def compute_rg_flow_loss(phi_net, rg_flow_gift, geodesic_integrator, config: Dict,
                         coords: torch.Tensor, torsion: torch.Tensor, epoch: int):
    """
    UPDATED v1.1c: Compute RG flow constraint loss using complete GIFT 2.1 formula.

    Uses RGFlowGIFT class to compute all four components.

    Args:
        phi_net: Neural network
        rg_flow_gift: RGFlowGIFT calculator
        geodesic_integrator: Geodesic integrator
        config: Configuration
        coords: Sample coordinates
        torsion: Torsion tensor (dphi)
        epoch: Current epoch

    Returns:
        loss_rg: RG flow loss
        components: Component breakdown dict
    """
    # Compute Δα using complete GIFT 2.1 formula
    delta_alpha, components = rg_flow_gift.compute_delta_alpha(
        phi_net, geodesic_integrator.geometry, coords, torsion, epoch
    )

    # Loss: match target Δα
    target_delta = config['rg_flow']['target_delta_alpha']
    loss_rg = (delta_alpha - target_delta) ** 2

    # Store in components for monitoring
    components['loss'] = loss_rg.item()
    components['target'] = target_delta

    return loss_rg, components

print("RG flow loss function updated to use RGFlowGIFT (GIFT 2.1 complete)")


RG flow loss function updated to use RGFlowGIFT (GIFT 2.1 complete)


In [ ]:
def compute_torsion_targeting_loss(dphi: torch.Tensor, dpsi_norm: torch.Tensor,
                                   target_torsion: float, config: Dict) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    NEW v1.1: Target torsion magnitude rather than minimize to zero.

    This is the CRITICAL FIX for v1.1:
    - v1.0f minimized: loss = ||dφ||² + ||dψ||² → drives torsion to machine precision
    - v1.1 targets: loss = (||T|| - target)² → maintains phenomenologically viable torsion

    Args:
        dphi: Exterior derivative tensor (batch, 7, 7, 7, 7)
        dpsi_norm: Scalar norm of d*φ
        target_torsion: Target ||T|| from GIFT calibration (phase-dependent)
        config: Configuration dictionary

    Returns:
        loss_torsion: Squared deviation from target
        actual_torsion: Current torsion magnitude (for monitoring)
    """
    # Compute actual torsion norm from dphi and dpsi
    dphi_norm = torch.sqrt((dphi ** 2).sum(dim=(-4,-3,-2,-1)).mean())
    actual_torsion = torch.sqrt(dphi_norm**2 + dpsi_norm**2)

    # Penalize deviation from target (not from zero)
    loss_torsion = (actual_torsion - target_torsion)**2

    # Prevent collapse to zero (floor enforcement)
    torsion_floor = config.get('torsion_floor', 1e-9)
    if actual_torsion < torsion_floor:
        loss_torsion += 100.0 * (torsion_floor - actual_torsion)

    return loss_torsion, actual_torsion


print("Torsion targeting loss ready (v1.1 critical fix)")


Torsion targeting loss ready (v1.1 critical fix)


## 7. NEW v1.1: AlphaInverseFunctional (Observable-Based Training)


In [ ]:
class AlphaInverseFunctional(nn.Module):
    """
    NEW v1.1: Compute α⁻¹ as functional of geometry.

    α⁻¹(x) = α⁻¹_ref + A·δ(det g) + B·δ||T|| + Σ C_i·δT^{components}

    This functional explicitly depends on torsion magnitude, creating physics-based
    constraint that maintains ||T|| ≠ 0 for phenomenological viability.
    """

    def __init__(self, config: Dict):
        super().__init__()

        # Reference values (M_Z scale)
        self.alpha_inv_ref = 137.036  # QED fine structure constant
        self.det_g_ref = config['target']['det_g']
        self.T_norm_ref = config['target']['torsion_norm']

        # Learnable coefficients (calibrated globally at epoch 6000)
        # Initial values from toy model validation
        self.A = nn.Parameter(torch.tensor(-4.68))  # det(g) coefficient
        self.B = nn.Parameter(torch.tensor(15.17))  # torsion norm coefficient
        self.C = nn.Parameter(torch.tensor([10.0, 5.0, 1.0]))  # component coefficients

    def forward(self, g: torch.Tensor, T: torch.Tensor) -> torch.Tensor:
        """
        Args:
            g: Metric tensor (batch, 7, 7)
            T: Torsion tensor (batch, 7, 7, 7) - this is dphi in our case

        Returns:
            alpha_inv: (batch,) - fine structure constant at each point
        """
        det_g = torch.linalg.det(g)
        T_norm = torch.sqrt((T**2).sum(dim=(-3,-2,-1)))

        # Key torsion components (indices: electron, pion, photon)
        # These specific components couple to fermion mass generation
        T_components = torch.stack([
            T[:, 1, 0, 2],  # T^π_{eφ}
            T[:, 0, 1, 2],  # T^e_{πφ}
            T[:, 2, 0, 1]   # T^φ_{eπ}
        ], dim=-1)

        # Deviations from reference
        delta_det_g = det_g - self.det_g_ref
        delta_T_norm = T_norm - self.T_norm_ref
        delta_T_components = T_components  # Already relative

        # Functional
        alpha_inv = self.alpha_inv_ref
        alpha_inv = alpha_inv + self.A * delta_det_g
        alpha_inv = alpha_inv + self.B * delta_T_norm
        alpha_inv = alpha_inv + torch.sum(self.C * delta_T_components, dim=-1)

        return alpha_inv


alpha_functional = AlphaInverseFunctional(CONFIG).to(device)
print(f"AlphaInverseFunctional initialized")
print(f"  Reference α⁻¹ = {alpha_functional.alpha_inv_ref}")
print(f"  Initial coefficients: A={alpha_functional.A.item():.2f}, B={alpha_functional.B.item():.2f}")
print(f"  Component weights: C={alpha_functional.C.data.cpu().numpy()}")


AlphaInverseFunctional initialized
  Reference α⁻¹ = 137.036
  Initial coefficients: A=-4.68, B=15.17
  Component weights: C=[10.  5.  1.]


## 8. NEW v1.1: Geodesic Integrator for RG Flow


In [111]:
class GeodesicIntegrator:
    """
    NEW v1.1: RK4/5 integrator with cached Christoffel symbols for geodesic integration.

    Integrates geodesic equation: d²x/dλ² + Γ^i_jk (dx/dλ)^j (dx/dλ)^k = 0

    Used for RG flow: integrate from M_Z (initial) to M_Planck (final) along
    quasi-static trajectory in moduli space.
    """

    def __init__(self, phi_net: nn.Module, geometry: TCSGeometry, config: Dict):
        self.phi_net = phi_net
        self.geometry = geometry
        self.config = config
        self.christoffel_cache = {}  # Cache for computed Christoffel symbols

    def compute_christoffel(self, x: torch.Tensor, g: torch.Tensor, g_inv: torch.Tensor) -> torch.Tensor:
        """
        Compute Christoffel symbols Γ^i_jk = (1/2) g^il (∂_j g_lk + ∂_k g_jl - ∂_l g_jk).

        Args:
            x: Position (batch, 7)
            g: Metric (batch, 7, 7)
            g_inv: Inverse metric (batch, 7, 7)

        Returns:
            christoffel: (batch, 7, 7, 7) - Γ^i_jk
        """
        batch_size = x.shape[0]
        christoffel = torch.zeros(batch_size, 7, 7, 7, device=x.device, dtype=x.dtype)

        epsilon = 1e-4

        # Compute metric derivatives numerically
        for l in range(7):
            x_plus = x.clone()
            x_plus[:, l] += epsilon
            x_plus[:, l] = x_plus[:, l] % 1.0

            x_minus = x.clone()
            x_minus[:, l] -= epsilon
            x_minus[:, l] = x_minus[:, l] % 1.0

            with torch.no_grad():
                phi_plus = self.phi_net(x_plus)
                phi_minus = self.phi_net(x_minus)

                g_plus, _ = compute_g2_metric(phi_plus, phase=5)
                g_minus, _ = compute_g2_metric(phi_minus, phase=5)

                g_plus = self.geometry.acyl_metric_correction(x_plus, g_plus)
                g_minus = self.geometry.acyl_metric_correction(x_minus, g_minus)

                dg_dl = (g_plus - g_minus) / (2 * epsilon)

            # Compute Christoffel components: Γ^i_jk = (1/2) Σ_l g^il (∂_j g_lk + ∂_k g_jl - ∂_l g_jk)
            # Here dg_dl is ∂_l g
            for i in range(7):
                for j in range(7):
                    for k in range(7):
                        # This term contributes to Γ^i_jk via: (1/2) g^il * (-∂_l g_jk)
                        val = -0.5 * dg_dl[:, j, k]  # shape: (batch,)
                        christoffel[:, i, j, k] += g_inv[:, i, l] * val

            # Now add the ∂_j g_lk and ∂_k g_jl terms
            # When we have ∂_l g computed, and we want ∂_j g_lk, we need the l-th derivative evaluated at index [l,k]
            # But we need to compute this differently - we need all derivatives

        # Recompute with proper derivative structure
        # We need ∂_j g_mn for all j,m,n
        dg = torch.zeros(batch_size, 7, 7, 7, device=x.device, dtype=x.dtype)  # dg[b, j, m, n] = ∂_j g_mn

        for j in range(7):
            x_plus = x.clone()
            x_plus[:, j] += epsilon
            x_plus[:, j] = x_plus[:, j] % 1.0

            x_minus = x.clone()
            x_minus[:, j] -= epsilon
            x_minus[:, j] = x_minus[:, j] % 1.0

            with torch.no_grad():
                phi_plus = self.phi_net(x_plus)
                phi_minus = self.phi_net(x_minus)

                g_plus, _ = compute_g2_metric(phi_plus, phase=5)
                g_minus, _ = compute_g2_metric(phi_minus, phase=5)

                g_plus = self.geometry.acyl_metric_correction(x_plus, g_plus)
                g_minus = self.geometry.acyl_metric_correction(x_minus, g_minus)

                dg[:, j, :, :] = (g_plus - g_minus) / (2 * epsilon)

        # Now compute Christoffel: Γ^i_jk = (1/2) Σ_l g^il (∂_j g_lk + ∂_k g_jl - ∂_l g_jk)
        christoffel = torch.zeros(batch_size, 7, 7, 7, device=x.device, dtype=x.dtype)
        for i in range(7):
            for j in range(7):
                for k in range(7):
                    for l in range(7):
                        term = 0.5 * (dg[:, j, l, k] + dg[:, k, j, l] - dg[:, l, j, k])
                        christoffel[:, i, j, k] += g_inv[:, i, l] * term

        return christoffel

    def geodesic_rhs(self, x: torch.Tensor, v: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Right-hand side of geodesic equation.

        dx/dλ = v
        dv/dλ = -Γ^i_jk v^j v^k

        Args:
            x: Position (batch, 7)
            v: Velocity (batch, 7)

        Returns:
            dx/dλ, dv/dλ
        """
        with torch.no_grad():
            phi = self.phi_net(x)
            g, g_inv = compute_g2_metric(phi, phase=5)
            g = self.geometry.acyl_metric_correction(x, g)
            g = normalize_metric(g, self.config['target']['det_g'])

            christoffel = self.compute_christoffel(x, g, g_inv)

            # dv/dλ = -Γ^i_jk v^j v^k
            dv = -torch.einsum('bijk,bj,bk->bi', christoffel, v, v)

        return v, dv

    def integrate_rk4(self, x0: torch.Tensor, v0: torch.Tensor, lambda_max: float, n_steps: int) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        RK4 integration of geodesic from λ=0 to λ=lambda_max.

        Args:
            x0: Initial position (7,)
            v0: Initial velocity (7,)
            lambda_max: Maximum affine parameter (ln(M_Planck/M_Z) = 39.44)
            n_steps: Number of integration steps

        Returns:
            x_trajectory: (n_steps+1, 7) - positions along geodesic
            v_trajectory: (n_steps+1, 7) - velocities along geodesic
        """
        dlambda = lambda_max / n_steps

        x_traj = [x0.unsqueeze(0)]
        v_traj = [v0.unsqueeze(0)]

        x = x0.unsqueeze(0)
        v = v0.unsqueeze(0)

        for step in range(n_steps):
            # RK4 integration
            k1_x, k1_v = self.geodesic_rhs(x, v)

            k2_x, k2_v = self.geodesic_rhs(x + 0.5 * dlambda * k1_x, v + 0.5 * dlambda * k1_v)

            k3_x, k3_v = self.geodesic_rhs(x + 0.5 * dlambda * k2_x, v + 0.5 * dlambda * k2_v)

            k4_x, k4_v = self.geodesic_rhs(x + dlambda * k3_x, v + dlambda * k3_v)

            # Update
            x = x + (dlambda / 6.0) * (k1_x + 2*k2_x + 2*k3_x + k4_x)
            v = v + (dlambda / 6.0) * (k1_v + 2*k2_v + 2*k3_v + k4_v)

            # Periodic boundary conditions
            x = x % 1.0

            x_traj.append(x)
            v_traj.append(v)

        x_trajectory = torch.cat(x_traj, dim=0)
        v_trajectory = torch.cat(v_traj, dim=0)

        return x_trajectory, v_trajectory


geodesic_integrator = GeodesicIntegrator(phi_net, geometry, CONFIG)
print("Geodesic integrator ready (RK4 with Christoffel computation)")

Geodesic integrator ready (RK4 with Christoffel computation)


## 9. NEW v1.1: RG Flow Loss and Calibration


In [ ]:
# NOTE: Legacy v1.1 RG flow functions removed for v1.1c
# v1.1c uses RGFlowGIFT class (Cell 18) with GIFT 2.1 complete formula
# instead of AlphaInverseFunctional approach

print("v1.1c: Using RGFlowGIFT for complete GIFT 2.1 RG flow")


v1.1c: Using RGFlowGIFT for complete GIFT 2.1 RG flow


## 10. Discrete Laplacian and Live Harmonic Extraction


In [ ]:
class DiscreteLaplacian:
    def __init__(self, n_grid: int, dim: int):
        self.n = n_grid
        self.dim = dim
        self.dx = 1.0 / n_grid

        from scipy.special import comb
        self.n_dof = int(comb(7, dim)) * (n_grid ** 7)

        print(f"Building {dim}-form Laplacian: {self.n}^7 = {self.n**7:,} points...")
        self.laplacian = self._build_laplacian_fast()

    def _build_laplacian_fast(self) -> csr_matrix:
        """Vectorized Laplacian construction."""
        n = self.n
        n_total = n ** 7

        print(f"  Allocating sparse structure for {n_total:,} × {n_total:,} matrix...")

        # Pre-allocate arrays for COO format
        max_entries = n_total * (1 + 2 * 7)
        row_indices = []
        col_indices = []
        data = []

        stencil = -2.0 * 7 / (self.dx ** 2)
        neighbor = 1.0 / (self.dx ** 2)

        # Diagonal entries
        print(f"  Building diagonal...")
        diag_idx = np.arange(n_total)
        row_indices.append(diag_idx)
        col_indices.append(diag_idx)
        data.append(np.full(n_total, stencil))

        # Off-diagonal entries (vectorized by axis)
        print(f"  Building off-diagonal entries...")
        for axis in range(7):
            if axis % 2 == 0:
                print(f"    Processing axis {axis+1}/7...")

            # Compute all indices at once
            all_coords = np.array(np.unravel_index(np.arange(n_total), [n] * 7))

            # Forward neighbors
            coords_plus = all_coords.copy()
            coords_plus[axis] = (coords_plus[axis] + 1) % n
            idx_plus = np.ravel_multi_index(coords_plus, [n] * 7)

            row_indices.append(diag_idx)
            col_indices.append(idx_plus)
            data.append(np.full(n_total, neighbor))

            # Backward neighbors
            coords_minus = all_coords.copy()
            coords_minus[axis] = (coords_minus[axis] - 1) % n
            idx_minus = np.ravel_multi_index(coords_minus, [n] * 7)

            row_indices.append(diag_idx)
            col_indices.append(idx_minus)
            data.append(np.full(n_total, neighbor))

        # Concatenate all arrays
        print(f"  Assembling sparse matrix...")
        row_indices = np.concatenate(row_indices)
        col_indices = np.concatenate(col_indices)
        data = np.concatenate(data)

        # Build COO then convert to CSR
        from scipy.sparse import coo_matrix
        L = coo_matrix((data, (row_indices, col_indices)), shape=(n_total, n_total))
        print(f"  Converting to CSR format...")
        L_csr = L.tocsr()
        print(f"  ✓ Laplacian built: {L_csr.nnz:,} non-zero entries")

        return L_csr

    def compute_spectrum(self, k: int = 100) -> Tuple[np.ndarray, np.ndarray]:
        """Extract k smallest eigenvalues/vectors."""
        print(f"  Computing {k} smallest eigenmodes (this may take 5-15 min)...")
        import time
        start = time.time()

        # Limit k to avoid convergence issues
        k_safe = min(k, self.laplacian.shape[0] - 10)

        eigenvalues, eigenvectors = eigsh(self.laplacian, k=k_safe, which='SM',
                                          tol=1e-5, maxiter=1000)

        elapsed = time.time() - start
        print(f"  ✓ Spectrum computed in {elapsed/60:.1f} minutes")

        return eigenvalues, eigenvectors


class LiveHarmonicExtractor:
    def __init__(self, laplacian: DiscreteLaplacian, target_dim: int, threshold: float = 1e-6):
        self.laplacian = laplacian
        self.target_dim = target_dim
        self.threshold = threshold

    def extract(self) -> Tuple[np.ndarray, int]:
        """Extract harmonic modes and return (modes, effective_dimension)."""
        k = min(self.target_dim + 50, self.laplacian.laplacian.shape[0] - 10)

        eigenvalues, eigenvectors = self.laplacian.compute_spectrum(k=k)

        print(f"  Eigenvalue range: [{eigenvalues[0]:.2e}, {eigenvalues[-1]:.2e}]")

        harmonic_mask = np.abs(eigenvalues) < self.threshold
        harmonic_indices = np.where(harmonic_mask)[0]

        print(f"  Found {len(harmonic_indices)} harmonic modes (threshold={self.threshold:.2e})")

        if len(harmonic_indices) < self.target_dim:
            print(f"  ⚠ Using {self.target_dim} smallest modes instead")
            harmonic_indices = np.arange(min(self.target_dim, len(eigenvalues)))
        else:
            harmonic_indices = harmonic_indices[:self.target_dim]

        harmonic_modes = eigenvectors[:, harmonic_indices]

        Q, R = np.linalg.qr(harmonic_modes)

        b_eff = len(harmonic_indices)

        return Q, b_eff

print("Discrete Laplacian and live harmonic extractor ready (optimized version)")


Discrete Laplacian and live harmonic extractor ready (optimized version)


## 11. Complete Loss Function with Torsion Targeting


In [ ]:
def compute_complete_loss(phi_net: nn.Module, coords: torch.Tensor, geometry: TCSGeometry,
                         extd: ExteriorDerivative, config: Dict, phase_weights: Dict,
                         rg_flow_gift: Optional[RGFlowGIFT] = None,
                         geodesic_integrator: Optional[GeodesicIntegrator] = None,
                         harmonic_extractors: Optional[Dict] = None, phase: int = 1) -> Dict[str, torch.Tensor]:
    """
    NEW v1.1: Complete multi-component loss with torsion targeting (not minimization).

    Key changes from v1.0f:
    - Torsion: target specific magnitude (phase-dependent) instead of minimize to zero
    - RG flow: add geodesic integration constraint in Phases 4-5
    - Early stopping: phase-specific criteria for systematic convergence
    """
    w = phase_weights
    target = config['target']
    target_torsion = config['torsion_targets'][phase]

    phi = phi_net(coords)
    g, g_inv = compute_g2_metric(phi, phase=phase)

    g = geometry.acyl_metric_correction(coords, g)
    g = normalize_metric(g, target['det_g'])

    psi = compute_hodge_dual(phi, g, g_inv)

    jacobian = extd.compute_jacobian(phi_net, coords)
    dphi = extd.d_phi(jacobian)
    dpsi_norm = extd.d_psi_norm(psi, phi_net, coords)

    # NEW v1.1: Torsion targeting (critical fix)
    loss_torsion, actual_torsion = compute_torsion_targeting_loss(dphi, dpsi_norm, target_torsion, config)

    # Determinant constraint
    det_g = torch.linalg.det(g)
    loss_det = ((det_g - target['det_g']) ** 2).mean()

    # Positivity with robust eigenvalue computation
    try:
        g_sym = (g + g.transpose(-2, -1)) / 2.0
        floor_eps = {1: 2e-3, 2: 1.5e-3, 3: 1e-3, 4: 5e-4, 5: 1e-4}
        g_sym = g_sym + floor_eps.get(phase, 1e-5) * torch.eye(7, device=g.device).unsqueeze(0)
        eigvals = torch.linalg.eigvalsh(g_sym)
        loss_positivity = torch.relu(-eigvals.min(dim=-1)[0]).mean()
    except:
        g_regularized = g + 1e-2 * torch.eye(7, device=g.device).unsqueeze(0)
        g_regularized = (g_regularized + g_regularized.transpose(-2, -1)) / 2.0
        try:
            eigvals = torch.linalg.eigvalsh(g_regularized)
            loss_positivity = torch.relu(-eigvals.min(dim=-1)[0]).mean()
        except:
            eigvals = torch.ones(g.shape[0], 7, device=g.device) * 0.1
            loss_positivity = torch.tensor(100.0, device=g.device)

    eig_floor_threshold = {1: 3e-3, 2: 2e-3, 3: 1e-3, 4: 1e-4, 5: 1e-5}
    threshold = eig_floor_threshold.get(phase, 1e-4)
    loss_eig_floor = torch.relu(threshold - eigvals.min(dim=-1)[0]).mean()

    r = geometry.radial_coordinate(coords)
    regions = geometry.region_classification(r)

    # Neck matching
    loss_neck_match = torch.tensor(0.0, device=coords.device)
    if regions['Neck'].any():
        coords_neck = coords[regions['Neck']]
        coords_twisted = geometry.twist_map(coords_neck)

        phi_neck = phi_net(coords_neck)
        phi_twisted = phi_net(coords_twisted)

        loss_neck_match = ((phi_neck - phi_twisted) ** 2).mean()

    # ACyl matching
    loss_acyl = torch.tensor(0.0, device=coords.device)
    if regions['M1'].any() or regions['M2'].any():
        r_acyl = torch.cat([r[regions['M1']], r[regions['M2']]]) if regions['M1'].any() and regions['M2'].any() else (r[regions['M1']] if regions['M1'].any() else r[regions['M2']])
        F = geometry.acyl.F(r_acyl)
        H = geometry.acyl.H(r_acyl)

        loss_acyl = ((F - 1.0) ** 2).mean() + (H ** 2).mean()

        loss_acyl_derivatives = geometry.compute_normal_derivative_mismatch(phi_net, coords, extd)
        loss_acyl = loss_acyl + loss_acyl_derivatives

    # Harmonicity with robust SVD
    loss_harmonicity = torch.tensor(0.0, device=coords.device)
    if harmonic_extractors is not None and w['harmonicity'] > 0:
        sample_size = min(256, coords.shape[0])
        coords_sample = coords[:sample_size]

        phi_sample = phi_net(coords_sample)
        phi_flat = phi_sample.flatten(start_dim=1)

        # Add small regularization to avoid SVD issues
        phi_flat = phi_flat + 1e-8 * torch.randn_like(phi_flat)

        target_rank_2 = target['b2']

        try:
            U, S, Vh = torch.linalg.svd(phi_flat, full_matrices=False)

            if S.shape[0] > target_rank_2:
                loss_harmonicity = loss_harmonicity + S[target_rank_2:].pow(2).sum()

            eigenvalue_penalty = torch.tensor(0.0, device=coords.device)
            if len(S) >= target_rank_2:
                eigenvalue_penalty = torch.relu(config['torsion_threshold'] - S[:target_rank_2].min())

            loss_harmonicity = loss_harmonicity + 10.0 * eigenvalue_penalty
        except Exception as e:
            # If SVD fails, use a simple L2 penalty instead
            print(f"SVD failed in harmonicity, using L2 penalty fallback")
            loss_harmonicity = 0.001 * (phi_flat ** 2).sum()

    # UPDATED v1.1c: RG flow loss with adaptive frequency
    loss_rg = torch.tensor(0.0, device=coords.device)
    rg_components = {}
    if phase >= 4 and w['rg_flow'] > 0 and rg_flow_gift is not None:
        # Adaptive frequency based on torsion magnitude
        base_freq = config['rg_flow']['geodesic_batch_freq_base']
        if config['rg_flow']['adaptive_frequency']:
            T_magnitude = torch.norm(dphi)
            adaptive_factor = 1.0 + 0.5 * torch.tanh(T_magnitude / 0.01)
            freq = torch.clamp(base_freq * adaptive_factor, 0.1, 0.8).item()
        else:
            freq = base_freq

        # Compute RG flow stochastically
        if torch.rand(1).item() < freq:
            loss_rg, rg_components = compute_rg_flow_loss(
                phi_net, rg_flow_gift, geodesic_integrator, config,
                coords, dphi, phase
            )

    w_eig_floor = {1: 1.5, 2: 0.8, 3: 0.3, 4: 0.05, 5: 0.01}
    eig_weight = w_eig_floor.get(phase, 0.1)

    total_loss = (w['torsion'] * loss_torsion +
                  w['det'] * loss_det +
                  w['positivity'] * loss_positivity +
                  w['neck_match'] * loss_neck_match +
                  w['acyl'] * loss_acyl +
                  w['harmonicity'] * loss_harmonicity +
                  w['rg_flow'] * loss_rg +
                  eig_weight * loss_eig_floor)

    # NaN guard
    if torch.isnan(total_loss) or torch.isinf(total_loss):
        print(f"NaN detected! Phase {phase}")
        total_loss = torch.tensor(1000.0, device=coords.device, requires_grad=True)

    return {
        'total': total_loss,
        'torsion': loss_torsion,
        'actual_torsion': actual_torsion,
        'det': loss_det,
        'positivity': loss_positivity,
        'neck_match': loss_neck_match,
        'acyl': loss_acyl,
        'harmonicity': loss_harmonicity,
        'rg_flow': loss_rg,
        'rg_components': rg_components,
        'eig_floor': loss_eig_floor
    }

print("Complete loss function ready (v1.1 with torsion targeting and RG flow)")

Complete loss function ready (v1.1 with torsion targeting and RG flow)


## 12. Checkpoint Manager


In [ ]:
class CheckpointManager:
    def __init__(self, config: Dict):
        self.checkpoint_dir = Path(config['checkpoint_dir'])
        self.checkpoint_dir.mkdir(exist_ok=True)
        self.freq = config['checkpoint_freq']

    def save(self, phase: int, epoch: int, model: nn.Module, optimizer: optim.Optimizer,
             loss_history: list, metadata: Dict):
        checkpoint = {
            'phase': phase,
            'epoch': epoch,
            'model_state': model.state_dict(),
            'optimizer_state': optimizer.state_dict(),
            'loss_history': loss_history,
            'metadata': metadata
        }

        path = self.checkpoint_dir / f'checkpoint_phase{phase}_epoch_{epoch}.pt'
        torch.save(checkpoint, path)

        latest_path = self.checkpoint_dir / 'checkpoint_latest.pt'
        torch.save(checkpoint, latest_path)

        config_path = self.checkpoint_dir / 'config.json'
        with open(config_path, 'w') as f:
            json.dump(metadata.get('config', {}), f, indent=2)

    def load_latest(self) -> Optional[Dict]:
        latest_path = self.checkpoint_dir / 'checkpoint_latest.pt'

        if latest_path.exists():
            checkpoint = torch.load(latest_path, map_location=device)
            return checkpoint

        return None

    def should_save(self, epoch: int) -> bool:
        return (epoch + 1) % self.freq == 0

checkpoint_mgr = CheckpointManager(CONFIG)
print(f"Checkpoint manager: {checkpoint_mgr.checkpoint_dir}")


Checkpoint manager: checkpoints_v1_1c


## 13. NEW v1.1: Multi-Phase Training with Systematic Early Stopping


In [ ]:
def check_phase_early_stop(losses_dict: Dict, phase_config: Dict, phase_history: list, config: Dict) -> bool:
    """
    NEW v1.1: Check phase-specific early stopping criteria.

    Each phase has its own convergence criteria to prevent over-optimization
    that could degrade subsequent phases.

    Args:
        losses_dict: Current loss values
        phase_config: Phase configuration with early_stop criteria
        phase_history: Recent loss history for this phase
        config: Global configuration

    Returns:
        should_stop: Whether to stop this phase early
    """
    if 'early_stop' not in phase_config:
        return False

    early_stop_config = phase_config['early_stop']
    patience = early_stop_config['patience']
    criteria = early_stop_config['criteria']

    # Need sufficient history
    if len(phase_history) < patience:
        return False

    # Check if all criteria satisfied for patience epochs
    recent_history = phase_history[-patience:]

    all_satisfied = True
    for criterion_name, threshold in criteria.items():
        if criterion_name == 'torsion_target_reached':
            # Check if torsion is within 20% of target
            if threshold:
                for hist_entry in recent_history:
                    actual_torsion = hist_entry.get('actual_torsion', 0)
                    phase = hist_entry.get('phase', 1)
                    target_torsion = config['torsion_targets'][phase]
                    error = abs(actual_torsion - target_torsion) / target_torsion
                    if error > 0.2:  # More than 20% error
                        all_satisfied = False
                        break
        else:
            # Standard threshold check
            for hist_entry in recent_history:
                if hist_entry.get(criterion_name, float('inf')) > threshold:
                    all_satisfied = False
                    break

        if not all_satisfied:
            break

    return all_satisfied


def train_multiphase(phi_net: nn.Module, geometry: TCSGeometry, extd: ExteriorDerivative,
                     config: Dict, checkpoint_mgr: CheckpointManager,
                     alpha_functional: AlphaInverseFunctional,
                     geodesic_integrator: GeodesicIntegrator):
    """
    NEW v1.1: Multi-phase curriculum training with:
    - Systematic per-phase early stopping
    - RG flow coefficient calibration at epoch 6000
    - Torsion targeting (not minimization)
    - Observable-based constraints
    """

    import time
    from IPython.display import display, HTML

    # UPDATED v1.1c: Use initial LR for phase 1
    optimizer = optim.AdamW(phi_net.parameters(), lr=config['learning_rate_phase12'], weight_decay=1e-5)

    # NEW v1.1c: Two-speed LR scheduler
    lr_scheduler = TwoSpeedLRScheduler(optimizer, config)

    start_phase = 1
    start_epoch = 0
    loss_history = []

    checkpoint = checkpoint_mgr.load_latest()
    if checkpoint is not None:
        phi_net.load_state_dict(checkpoint['model_state'])
        optimizer.load_state_dict(checkpoint['optimizer_state'])
        start_phase = checkpoint['phase']
        start_epoch = checkpoint['epoch'] + 1
        loss_history = checkpoint['loss_history']
        print(f"Resumed from Phase {checkpoint['phase']}, Epoch {checkpoint['epoch']}")

    n_epochs_per_phase = config['n_epochs_per_phase']
    batch_size = config['batch_size']

    harmonic_extractors = {}

    for phase in range(start_phase, len(config['phases']) + 1):
        phase_config = config['phases'][phase]
        print(f"\n{'='*60}")
        print(f"PHASE {phase}: {phase_config['name']}")
        print(f"Torsion target: ||T|| = {config['torsion_targets'][phase]}")
        print(f"{'='*60}")

        if phase >= 3:
            harmonic_extractors = {'enabled': True}

        epoch_start = start_epoch if phase == start_phase else 0

        phase_start_time = time.time()
        phase_history = []  # NEW v1.1: Track history for this phase
        patience_counter = 0

        for epoch in range(epoch_start, n_epochs_per_phase):
            # Compute global epoch for tracking
            global_epoch = sum([config['n_epochs_per_phase'] for p in range(1, phase)]) + epoch

            phi_net.train()

            coords = torch.rand(batch_size, 7, device=device)

            optimizer.zero_grad()

            losses = compute_complete_loss(
                phi_net, coords, geometry, extd, config,
                phase_config['weights'],
                rg_flow_gift=rg_flow_gift if phase >= 4 else None,
                geodesic_integrator=geodesic_integrator if phase >= 4 else None,
                harmonic_extractors=harmonic_extractors if phase >= 3 else None,
                phase=phase
            )

            losses['total'].backward()
            torch.nn.utils.clip_grad_norm_(phi_net.parameters(), 0.5)

            # UPDATED v1.1c: Step LR scheduler
            current_lr = lr_scheduler.step(global_epoch, phase)

            optimizer.step()

            loss_entry = {
                'phase': phase,
                'epoch': epoch,
                'global_epoch': global_epoch,
                'total': losses['total'].item(),
                'torsion': losses['torsion'].item(),
                'actual_torsion': losses['actual_torsion'].item(),  # NEW v1.1
                'det': losses['det'].item(),
                'positivity': losses['positivity'].item(),
                'neck_match': losses['neck_match'].item(),
                'acyl': losses['acyl'].item(),
                'harmonicity': losses['harmonicity'].item(),
                'rg_flow': losses['rg_flow'].item(),  # NEW v1.1
                'eig_floor': losses['eig_floor'].item()
            }
            loss_history.append(loss_entry)
            phase_history.append(loss_entry)

            if (epoch + 1) % 100 == 0:
                elapsed = time.time() - phase_start_time
                epochs_done = epoch + 1 - epoch_start
                epochs_remaining = n_epochs_per_phase - (epoch + 1)
                eta_seconds = (elapsed / epochs_done) * epochs_remaining if epochs_done > 0 else 0
                eta_minutes = eta_seconds / 60

                progress_pct = 100 * (epoch + 1) / n_epochs_per_phase
                bar_length = 30
                filled = int(bar_length * progress_pct / 100)
                bar = '█' * filled + '░' * (bar_length - filled)

                print(f"[{bar}] {progress_pct:.1f}% | "
                      f"Epoch {epoch+1}/{n_epochs_per_phase} | "
                      f"Loss={losses['total'].item():.6f} | "
                      f"||T||={losses['actual_torsion'].item():.4e} (target={config['torsion_targets'][phase]:.4e}) | "
                      f"LR={current_lr:.2e} | "
                      f"RG={losses['rg_flow'].item():.4e} | "
                      f"ETA: {eta_minutes:.1f}m")

            # NEW v1.1: Per-phase early stopping
            if check_phase_early_stop(losses, phase_config, phase_history, config):
                patience_counter += 1
                if patience_counter >= phase_config['early_stop']['patience']:
                    print(f"\n{'='*60}")
                    print(f"Early stopping Phase {phase} at epoch {epoch+1}")
                    print(f"All convergence criteria satisfied for {patience_counter} epochs")
                    print(f"{'='*60}\n")

                    metadata = {
                        'config': config,
                        'phase': phase,
                        'early_stopped': True,
                        'final_loss': losses['total'].item(),
                        'final_torsion': losses['actual_torsion'].item()
                    }
                    checkpoint_mgr.save(phase, epoch, phi_net, optimizer, loss_history, metadata)
                    break
            else:
                patience_counter = 0

            if checkpoint_mgr.should_save(epoch):
                metadata = {
                    'config': config,
                    'current_phase': phase,
                    'final_loss': losses['total'].item()
                }
                checkpoint_mgr.save(phase, epoch, phi_net, optimizer, loss_history, metadata)
                print(f"✓ Checkpoint saved")

        start_epoch = 0

    return loss_history

print("Multi-phase training pipeline ready (v1.1 with systematic early stopping)")


Multi-phase training pipeline ready (v1.1 with systematic early stopping)


## 14. Execute Training


In [ ]:
print("Starting multi-phase training (v1.1)...")
print(f"Extended neck: σ_neck={CONFIG['tcs']['neck_width']}")
print(f"Torsion targeting: {CONFIG['torsion_targets']}")
print(f"RG flow calibration: epoch {CONFIG['rg_flow']['calibration_epoch']}")
print("")

loss_history = train_multiphase(phi_net, geometry, extd, CONFIG, checkpoint_mgr, alpha_functional, geodesic_integrator)
print(f"\nTraining complete. Final loss: {loss_history[-1]['total']:.6f}")
print(f"Final torsion: ||T|| = {loss_history[-1]['actual_torsion']:.6e} (target: {CONFIG['target']['torsion_norm']:.6e})")


Starting multi-phase training (v1.1)...
Extended neck: σ_neck=5.0
Torsion targeting: {1: 0.001, 2: 0.005, 3: 0.01, 4: 0.015, 5: 0.0164}
RG flow calibration: epoch 2000

Resumed from Phase 4, Epoch 1999

PHASE 4: Harmonic_Extraction
Torsion target: ||T|| = 0.015

PHASE 5: RG_Calibration
Torsion target: ||T|| = 0.0164
[█░░░░░░░░░░░░░░░░░░░░░░░░░░░░░] 5.0% | Epoch 100/2000 | Loss=0.363781 | ||T||=2.5230e-02 (target=1.6400e-02) | LR=2.98e-04 | RG=0.0000e+00 | ETA: 70.8m
[███░░░░░░░░░░░░░░░░░░░░░░░░░░░] 10.0% | Epoch 200/2000 | Loss=1761.377930 | ||T||=2.1445e-02 (target=1.6400e-02) | LR=4.98e-04 | RG=3.5220e+04 | ETA: 68.7m
[████░░░░░░░░░░░░░░░░░░░░░░░░░░] 15.0% | Epoch 300/2000 | Loss=0.362330 | ||T||=1.5328e-02 (target=1.6400e-02) | LR=5.00e-04 | RG=0.0000e+00 | ETA: 66.5m
[██████░░░░░░░░░░░░░░░░░░░░░░░░] 20.0% | Epoch 400/2000 | Loss=0.361064 | ||T||=1.2822e-02 (target=1.6400e-02) | LR=4.99e-04 | RG=0.0000e+00 | ETA: 63.5m
[███████░░░░░░░░░░░░░░░░░░░░░░░] 25.0% | Epoch 500/2000 | Loss=0

## 15. Post-Training: Harmonic Extraction


In [ ]:
print("Building discrete Laplacians (reduced grid)...")
n_grid_harm = CONFIG['n_grid_harmonics']
print(f"  Using {n_grid_harm}^7 = {n_grid_harm**7:,} points (reduced from {CONFIG['n_grid']**7:,})")

laplacian_2 = DiscreteLaplacian(n_grid_harm, dim=2)
laplacian_3 = DiscreteLaplacian(n_grid_harm, dim=3)

print("Extracting harmonic 2-forms...")
extractor_2 = LiveHarmonicExtractor(laplacian_2, CONFIG['target']['b2'])
h2_modes, b2_eff = extractor_2.extract()
print(f"b₂_eff = {b2_eff} (target: {CONFIG['target']['b2']})")

print("Extracting harmonic 3-forms...")
extractor_3 = LiveHarmonicExtractor(laplacian_3, CONFIG['target']['b3'])
h3_modes, b3_eff = extractor_3.extract()
print(f"b₃_eff = {b3_eff} (target: {CONFIG['target']['b3']})")


Building discrete Laplacians (reduced grid)...
  Using 8^7 = 2,097,152 points (reduced from 268,435,456)
Building 2-form Laplacian: 8^7 = 2,097,152 points...
  Allocating sparse structure for 2,097,152 × 2,097,152 matrix...
  Building diagonal...
  Building off-diagonal entries...
    Processing axis 1/7...
    Processing axis 3/7...
    Processing axis 5/7...
    Processing axis 7/7...
  Assembling sparse matrix...
  Converting to CSR format...
  ✓ Laplacian built: 31,457,280 non-zero entries
Building 3-form Laplacian: 8^7 = 2,097,152 points...
  Allocating sparse structure for 2,097,152 × 2,097,152 matrix...
  Building diagonal...
  Building off-diagonal entries...
    Processing axis 1/7...
    Processing axis 3/7...
    Processing axis 5/7...
    Processing axis 7/7...
  Assembling sparse matrix...
  Converting to CSR format...
  ✓ Laplacian built: 31,457,280 non-zero entries
Extracting harmonic 2-forms...
  Computing 71 smallest eigenmodes (this may take 5-15 min)...
  ✓ Spectrum 

## 16. Yukawa Tensor Construction


In [109]:
class YukawaTensor:
    def __init__(self, h2_modes: np.ndarray, h3_modes: np.ndarray, n_samples: int):
        self.h2 = h2_modes
        self.h3 = h3_modes
        self.b2 = h2_modes.shape[1]
        self.b3 = h3_modes.shape[1]
        self.n_samples = n_samples

    def wedge_product(self, alpha: np.ndarray, beta: np.ndarray, gamma: np.ndarray) -> float:
        indices = np.random.randint(0, len(alpha), self.n_samples)
        integrand = alpha[indices] * beta[indices] * gamma[indices]
        return np.mean(integrand)

    def compute(self) -> np.ndarray:
        Y = np.zeros((self.b2, self.b2, self.b3))
        total = self.b2 * self.b2 * self.b3
        count = 0

        for alpha in range(self.b2):
            for beta in range(self.b2):
                for gamma in range(self.b3):
                    Y[alpha, beta, gamma] = self.wedge_product(
                        self.h2[:, alpha],
                        self.h2[:, beta],
                        self.h3[:, gamma]
                    )

                    count += 1
                    if count % 5000 == 0:
                        print(f"Yukawa progress: {count}/{total} ({100*count/total:.1f}%)")

        return Y

print(f"Computing Yukawa tensor ({b2_eff}×{b2_eff}×{b3_eff})...")
yukawa = YukawaTensor(h2_modes, h3_modes, CONFIG['yukawa_samples'])
Y_tensor = yukawa.compute()
print(f"Yukawa tensor shape: {Y_tensor.shape}")
print(f"Yukawa tensor norm: {np.linalg.norm(Y_tensor):.6e}")
print(f"NEW v1.1: Yukawa norm should be >>10⁻¹⁰ (v1.0f: ~10⁻¹⁰, expected v1.1: ~10⁻³ to 10⁻⁴)")


Computing Yukawa tensor (21×21×77)...
Yukawa progress: 5000/33957 (14.7%)
Yukawa progress: 10000/33957 (29.4%)
Yukawa progress: 15000/33957 (44.2%)
Yukawa progress: 20000/33957 (58.9%)
Yukawa progress: 25000/33957 (73.6%)
Yukawa progress: 30000/33957 (88.3%)
Yukawa tensor shape: (21, 21, 77)
Yukawa tensor norm: 5.904187e-10
NEW v1.1: Yukawa norm should be >>10⁻¹⁰ (v1.0f: ~10⁻¹⁰, expected v1.1: ~10⁻³ to 10⁻⁴)


## 17. NEW v1.1: Comprehensive Validation (Torsion + RG Flow)


In [113]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

print("="*60)
print("COMPREHENSIVE VALIDATION (v1.1)")
print("="*60)

# ==================================================================
# 1. TORSION MAGNITUDE VALIDATION (CRITICAL)
# ==================================================================
print("\n1. Torsion Magnitude Validation (v1.1 Critical Fix)")
print("-"*60)

phi_net.eval()
with torch.no_grad():
    n_samples = 10000
    test_coords = torch.rand(n_samples, 7, device=device)

    phi_vals = phi_net(test_coords)
    g_vals, g_inv = compute_g2_metric(phi_vals, phase=5)
    g_corrected = geometry.acyl_metric_correction(test_coords, g_vals)
    g_normalized = normalize_metric(g_corrected, CONFIG['target']['det_g'])

    # Compute Hodge dual for complete torsion
    psi_vals = compute_hodge_dual(phi_vals, g_normalized, g_inv)

    jacobian = extd.compute_jacobian(phi_net, test_coords)
    dphi = extd.d_phi(jacobian)
    dpsi_norm = extd.d_psi_norm(psi_vals, phi_net, test_coords)

    # Compute COMPLETE torsion per-point (dphi + dpsi)
    dphi_per_point = torch.sqrt((dphi ** 2).sum(dim=(-4, -3, -2, -1)))
    torsion_per_point = torch.sqrt(dphi_per_point**2 + dpsi_norm**2)

    torsion_mean = torsion_per_point.mean().item()
    torsion_max = torsion_per_point.max().item()
    torsion_std = torsion_per_point.std().item()

    det_g = torch.linalg.det(g_normalized)

target_torsion = CONFIG['target']['torsion_norm']
torsion_error = abs(torsion_mean - target_torsion) / target_torsion * 100

print(f"  Target torsion: ||T|| = {target_torsion:.6e}")
print(f"  Actual torsion: ||T|| = {torsion_mean:.6e}")
print(f"  Error: {torsion_error:.2f}%")
print(f"  Range: [{torsion_per_point.min().item():.6e}, {torsion_max:.6e}]")
print(f"  Std: {torsion_std:.6e}")

if torsion_error < 10:
    print(f"  ✓ Torsion within 10% of target (SUCCESS)")
elif torsion_error < 30:
    print(f"  ⚠ Torsion within 30% of target (ACCEPTABLE)")
else:
    print(f"  ✗ Torsion error too large (NEEDS IMPROVEMENT)")

print(f"\n  v1.0f comparison: ||T|| = 8.35×10⁻⁴ (94.9% error)")
print(f"  v1.1 improvement: {(torsion_mean / 8.35e-4):.1f}× closer to target")

# ==================================================================
# 2. RG FLOW VALIDATION (NEW)
# ==================================================================
print("\n2. RG Flow Validation (NEW v1.1)")
print("-"*60)

print("  Integrating geodesic from M_Z to M_Planck...")
x0 = torch.zeros(7, device=device)
v0 = torch.tensor([0, -1e-3, 0, 0, 0, 0, 0], device=device, dtype=torch.float32)

with torch.no_grad():
    x_traj, v_traj = geodesic_integrator.integrate_rk4(
        x0, v0,
        CONFIG['rg_flow']['lambda_max'],
        CONFIG['rg_flow']['n_integration_steps']
    )

    # Compute α⁻¹ at endpoints
    phi_start = phi_net(x_traj[0:1])
    g_start, _ = compute_g2_metric(phi_start, phase=5)
    g_start = geometry.acyl_metric_correction(x_traj[0:1], g_start)
    g_start = normalize_metric(g_start, CONFIG['target']['det_g'])

    jacobian_start = extd.compute_jacobian(phi_net, x_traj[0:1])
    dphi_start = extd.d_phi(jacobian_start)

    alpha_start = alpha_functional(g_start, dphi_start).mean()  # Take mean to get scalar

    phi_end = phi_net(x_traj[-1:])
    g_end, _ = compute_g2_metric(phi_end, phase=5)
    g_end = geometry.acyl_metric_correction(x_traj[-1:], g_end)
    g_end = normalize_metric(g_end, CONFIG['target']['det_g'])

    jacobian_end = extd.compute_jacobian(phi_net, x_traj[-1:])
    dphi_end = extd.d_phi(jacobian_end)

    alpha_end = alpha_functional(g_end, dphi_end).mean()  # Take mean to get scalar

    delta_alpha = (alpha_end - alpha_start).item()

target_delta = CONFIG['rg_flow']['target_delta_alpha']
rg_error = abs(delta_alpha - target_delta) / abs(target_delta) * 100

print(f"  α⁻¹(M_Z) = {alpha_start.item():.3f}")
print(f"  α⁻¹(M_Planck) = {alpha_end.item():.3f}")
print(f"  Δα⁻¹ = {delta_alpha:.4f}")
print(f"  Target: Δα⁻¹ = {target_delta:.4f}")
print(f"  Error: {rg_error:.2f}%")

if rg_error < 5:
    print(f"  ✓ RG flow within 5% of target (SUCCESS)")
elif rg_error < 10:
    print(f"  ⚠ RG flow within 10% of target (ACCEPTABLE)")
else:
    print(f"  ✗ RG flow error too large (NEEDS IMPROVEMENT)")
# ==================================================================
# 3. GEOMETRIC QUALITY (PRESERVED FROM v1.0f)
# ==================================================================
print("\n3. Geometric Quality (Should Be Preserved)")
print("-"*60)

g_sym = (g_normalized + g_normalized.transpose(-2, -1)) / 2.0
eigvals = torch.linalg.eigvalsh(g_sym)

det_mean = det_g.mean().item()
det_std = det_g.std().item()
det_error = abs(det_mean - CONFIG['target']['det_g']) / CONFIG['target']['det_g'] * 100

eig_min = eigvals.min().item()
positive_definite = (eigvals > 0).all().item()

print(f"  det(g) = {det_mean:.7f} ± {det_std:.2e}")
print(f"  Target: det(g) = {CONFIG['target']['det_g']}")
print(f"  Error: {det_error:.4f}%")
print(f"  Eigenvalues: min={eig_min:.6f}, max={eigvals.max().item():.6f}")
print(f"  Positive definite: {positive_definite}")

if det_error < 0.01 and positive_definite:
    print(f"  ✓ Geometric quality preserved (SUCCESS)")
else:
    print(f"  ⚠ Geometric quality degraded")

# ==================================================================
# 4. PHENOMENOLOGICAL VIABILITY
# ==================================================================
print("\n4. Phenomenological Viability")
print("-"*60)

yukawa_norm = np.linalg.norm(Y_tensor)
yukawa_max = np.abs(Y_tensor).max()
yukawa_nonzero_frac = (np.abs(Y_tensor) > 1e-8).mean()

print(f"  Yukawa norm: {yukawa_norm:.6e}")
print(f"  Max |Y|: {yukawa_max:.6e}")
print(f"  Non-zero fraction: {yukawa_nonzero_frac:.3f}")

if yukawa_norm > 1e-5:
    print(f"  ✓ Yukawa couplings non-negligible (SUCCESS)")
elif yukawa_norm > 1e-8:
    print(f"  ⚠ Yukawa couplings weak but non-zero (MARGINAL)")
else:
    print(f"  ✗ Yukawa couplings negligible (FAILURE)")

print(f"\n  v1.0f comparison: Yukawa norm ~10⁻¹⁰")
if yukawa_norm > 1e-10:
    print(f"  v1.1 improvement: {yukawa_norm / 1e-10:.1f}× larger")

# ==================================================================
# 5. SUMMARY
# ==================================================================
print("\n" + "="*60)
print("VALIDATION SUMMARY (v1.1)")
print("="*60)

all_tests_passed = (
    torsion_error < 10 and
    rg_error < 10 and
    det_error < 0.01 and
    positive_definite and
    yukawa_norm > 1e-5
)

print(f"Torsion target reached: {'✓' if torsion_error < 10 else '✗'}")
print(f"RG flow correct: {'✓' if rg_error < 10 else '✗'}")
print(f"Geometric quality preserved: {'✓' if (det_error < 0.01 and positive_definite) else '✗'}")
print(f"Yukawa viable: {'✓' if yukawa_norm > 1e-5 else '✗'}")
print(f"\nOverall: {'SUCCESS' if all_tests_passed else 'PARTIAL SUCCESS - Iteration may be needed'}")
print("="*60)


COMPREHENSIVE VALIDATION (v1.1)

1. Torsion Magnitude Validation (v1.1 Critical Fix)
------------------------------------------------------------
  Target torsion: ||T|| = 1.640000e-02
  Actual torsion: ||T|| = 1.822357e-02
  Error: 11.12%
  Range: [1.355493e-02, 2.914256e-02]
  Std: 2.422452e-03
  ⚠ Torsion within 30% of target (ACCEPTABLE)

  v1.0f comparison: ||T|| = 8.35×10⁻⁴ (94.9% error)
  v1.1 improvement: 21.8× closer to target

2. RG Flow Validation (NEW v1.1)
------------------------------------------------------------
  Integrating geodesic from M_Z to M_Planck...
  α⁻¹(M_Z) = 136.838
  α⁻¹(M_Planck) = 136.857
  Δα⁻¹ = 0.0184
  Target: Δα⁻¹ = -0.9000
  Error: 102.05%
  ✗ RG flow error too large (NEEDS IMPROVEMENT)

3. Geometric Quality (Should Be Preserved)
------------------------------------------------------------
  det(g) = 2.0000014 ± 6.36e-07
  Target: det(g) = 2.0
  Error: 0.0001%
  Eigenvalues: min=0.648077, max=1.941643
  Positive definite: True
  ✓ Geometric qualit

## 18. Save Results


In [115]:
output_dir = Path('outputs_v1_1')
output_dir.mkdir(exist_ok=True)

# Save harmonics and Yukawa
np.save(output_dir / 'harmonic_2forms.npy', h2_modes)
np.save(output_dir / 'harmonic_3forms.npy', h3_modes)
np.save(output_dir / 'yukawa_tensor.npy', Y_tensor)

# Save phi and metric samples
phi_net.eval()
with torch.no_grad():
    test_coords = torch.rand(5000, 7, device=device)
    phi_samples = phi_net(test_coords).cpu().numpy()
    np.save(output_dir / 'phi_samples.npy', phi_samples)

    g_samples, _ = compute_g2_metric(phi_net(test_coords))
    g_samples = geometry.acyl_metric_correction(test_coords, g_samples)
    np.save(output_dir / 'metric_samples.npy', g_samples.cpu().numpy())

# Save loss history
loss_df = pd.DataFrame(loss_history)
loss_df.to_csv(output_dir / 'training_history.csv', index=False)

# Save metadata with v1.1 specific info
metadata = {
    'version': '1.1',
    'config': CONFIG,
    'training_grid': f"{CONFIG['n_grid']}^7",
    'harmonics_grid': f"{CONFIG['n_grid_harmonics']}^7",
    'b2_effective': int(b2_eff),
    'b3_effective': int(b3_eff),
    'b2_target': CONFIG['target']['b2'],
    'b3_target': CONFIG['target']['b3'],
    'yukawa_shape': list(Y_tensor.shape),
    'yukawa_norm': float(yukawa_norm),
    'final_loss': loss_history[-1]['total'] if loss_history else None,
    'total_epochs': len(loss_history),
    'torsion_validation': {
        'target': float(target_torsion),
        'actual': float(torsion_mean),
        'error_percent': float(torsion_error),
        'passed': bool(torsion_error < 10)
    },
    'rg_flow_validation': {
        'target_delta_alpha': float(target_delta),
        'actual_delta_alpha': float(delta_alpha),
        'error_percent': float(rg_error),
        'passed': bool(rg_error < 10)
    },
    'geometric_validation': {
        'det_g_mean': float(det_mean),
        'det_g_error_percent': float(det_error),
        'positive_definite': bool(positive_definite),
        'passed': bool(det_error < 0.01 and positive_definite)
    },
    'phenomenological_validation': {
        'yukawa_norm': float(yukawa_norm),
        'passed': bool(yukawa_norm > 1e-5)
    },
    'note': 'v1.1: Torsion targeting + RG flow integration + Extended neck + Per-phase early stopping'
}

with open(output_dir / 'metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"\nResults saved to {output_dir}")
print(f"\nKey outputs:")
print(f"  - training_history.csv: Complete loss history with torsion tracking")
print(f"  - harmonic_2forms.npy, harmonic_3forms.npy: Cohomology bases")
print(f"  - yukawa_tensor.npy: ({b2_eff}×{b2_eff}×{b3_eff}) coupling tensor")
print(f"  - metadata.json: Validation results and configuration")
print(f"\n" + "="*60)
print("GIFT v1.1 PIPELINE COMPLETE")
print("="*60)
print(f"Torsion: {'✓' if torsion_error < 10 else '✗'} ({torsion_error:.1f}% error)")
print(f"RG flow: {'✓' if rg_error < 10 else '✗'} ({rg_error:.1f}% error)")
print(f"Geometry: {'✓' if (det_error < 0.01 and positive_definite) else '✗'}")
print(f"Yukawa: {'✓' if yukawa_norm > 1e-5 else '✗'} (norm={yukawa_norm:.2e})")
print("="*60)


Results saved to outputs_v1_1

Key outputs:
  - training_history.csv: Complete loss history with torsion tracking
  - harmonic_2forms.npy, harmonic_3forms.npy: Cohomology bases
  - yukawa_tensor.npy: (21×21×77) coupling tensor
  - metadata.json: Validation results and configuration

GIFT v1.1 PIPELINE COMPLETE
Torsion: ✗ (11.1% error)
RG flow: ✗ (102.0% error)
Geometry: ✓
Yukawa: ✗ (norm=5.90e-10)
